# Базовий фреймворк для задач задоволення обмежень

Базова модель задачі задоволення обмежень:

In [26]:
# csp.py
# From Classic Computer Science Problems in Python Chapter 3
# Copyright 2018 David Kopec
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type


# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...


# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")
        self.BT_calls = 0

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True
    
    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        self.BT_calls += 1
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None
    
    def get_unconstrained_values_count(self, assignment: Dict[V, D] = {}, variable: V = None) -> int:
        count = 0
        for value in self.domains[variable]:
            local_assignment = assignment.copy()
            local_assignment[variable] = value
            if self.consistent(variable, local_assignment):
                count += 1
        return count
    
    def MCV(self, assignment: Dict[V, D] = {}, unassigned: List[V] = {}) -> V:
        result = [self.get_unconstrained_values_count(assignment, variable) for variable in unassigned]
        return unassigned[result.index(min(result))]
    
    def LCV(self, assignment: Dict[V, D] = {}, variable: V = None) -> List[D]:
        values = []
        result = []
        unassigned: List[V] = [v for v in self.variables if v not in assignment and v != variable]
        for value in self.domains[variable]:
            local_assignment = assignment.copy()
            local_assignment[variable] = value
            if self.consistent(variable, local_assignment):
                values.append(value)
                count = 0
                for unassign in unassigned:
                    count += self.get_unconstrained_values_count(local_assignment, unassign)
                result.append(count)
        
        return [element for _, element in sorted(zip(result, values), key=lambda x: x[0], reverse=True)]
    
    def AC_3(self, assignment: Dict[V, D] = {}) -> bool:
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
        for variable in unassigned:
            if self.get_unconstrained_values_count(assignment, variable) == 0:
                return False
        return True

    # BT+MCV+LCV+AC-3
    def backtracking_search_method_1(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]: 
        self.BT_calls += 1

        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the most constrained value
        first: V = self.MCV(assignment, unassigned)
        for value in self.LCV(assignment, first):
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.AC_3(local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search_method_1(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None
    
    # BT+LCV
    def backtracking_search_method_2(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        self.BT_calls += 1
        
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.LCV(assignment, first):
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search_method_2(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None


# Задача з ферзями

Реалізація обмеження задачі:

In [27]:
class QueensConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]) -> None:
        super().__init__(columns)
        self.columns: List[int] = columns

    def satisfied(self, assignment: Dict[int, int]) -> bool:
        # q1c = queen 1 column, q1r = queen 1 row
        for q1c, q1r in assignment.items():
            # q2c = queen 2 column
            for q2c in range(q1c + 1, len(self.columns) + 1):
                if q2c in assignment:
                    q2r: int = assignment[q2c] # q2r = queen 2 row
                    if q1r == q2r: # same row?
                        return False
                    if abs(q1r - q2r) == abs(q1c - q2c): # same diagonal?
                        return False
        return True # no conflict

Рішення:

In [28]:
size: List[int] = [8, 12, 16, 20]


for n in size:
    columns: List[int] = [i for i in range(1, n + 1)]
    rows: Dict[int, List[int]] = {}
    for column in columns:
        rows[column] = [i for i in range(1, n + 1)]
    csp: CSP[int, int] = CSP(columns, rows)
    csp.add_constraint(QueensConstraint(columns))
    methods = [csp.backtracking_search, csp.backtracking_search_method_1, csp.backtracking_search_method_2]
    call_num = []
    for i in range(3):       
        csp.BT_calls = 0
        solution: Optional[Dict[int, int]] = methods[i]()
        if solution is None:
            print("No solution found!")
        else:
            print(solution)

        call_num.append(csp.BT_calls)
    print(n, call_num)

{1: 1, 2: 5, 3: 8, 4: 6, 5: 3, 6: 7, 7: 2, 8: 4}
{1: 1, 2: 6, 3: 8, 4: 3, 5: 7, 6: 4, 7: 2, 8: 5}
{1: 1, 2: 6, 3: 8, 4: 3, 5: 7, 6: 4, 7: 2, 8: 5}
8 [114, 30, 56]
{1: 1, 2: 3, 3: 5, 4: 8, 5: 10, 6: 12, 7: 6, 8: 11, 9: 2, 10: 7, 11: 9, 12: 4}
{1: 1, 2: 4, 3: 7, 8: 3, 6: 2, 10: 11, 9: 6, 5: 8, 4: 10, 12: 5, 7: 12, 11: 9}
{1: 1, 2: 4, 3: 7, 4: 10, 5: 12, 6: 5, 7: 8, 8: 11, 9: 3, 10: 6, 11: 2, 12: 9}
12 [262, 26, 259]
{1: 1, 2: 3, 3: 5, 4: 2, 5: 13, 6: 9, 7: 14, 8: 12, 9: 15, 10: 6, 11: 16, 12: 7, 13: 4, 14: 11, 15: 8, 16: 10}
{1: 1, 2: 4, 3: 7, 8: 16, 4: 10, 7: 2, 6: 9, 9: 3, 11: 8, 5: 12, 12: 11, 10: 15, 14: 6, 13: 14, 15: 13, 16: 5}
{1: 1, 2: 4, 3: 7, 4: 16, 5: 13, 6: 15, 7: 10, 8: 14, 9: 6, 10: 3, 11: 5, 12: 2, 13: 12, 14: 9, 15: 11, 16: 8}
16 [10053, 240, 2725]
{1: 1, 2: 3, 3: 5, 4: 2, 5: 4, 6: 13, 7: 15, 8: 12, 9: 18, 10: 20, 11: 17, 12: 9, 13: 16, 14: 19, 15: 8, 16: 10, 17: 7, 18: 14, 19: 6, 20: 11}
{1: 1, 2: 4, 3: 7, 8: 20, 4: 18, 7: 17, 9: 3, 12: 2, 15: 11, 10: 8, 5: 10, 11: 19, 6